In [1]:
import json

import evaluate
import torch as tt
import pandas as pd
import numpy as np

from typing import Any, Dict, Union

from nltk.tokenize import sent_tokenize
from tqdm import tqdm_notebook
from datasets import Dataset, load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, PreTrainedModel, PreTrainedTokenizer
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import EvalPrediction

In [2]:
# models:
tokenizer = T5Tokenizer.from_pretrained("ai-forever/ruT5-base")
model = T5ForConditionalGeneration.from_pretrained(
    "ai-forever/ruT5-base"
).to(tt.device("cuda:0"))

# metrics:
bleu4 = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/lib/python3/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
[nltk_data] Downloading package wordn

In [3]:
option_id_dict = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3
}

def to_new_format(example: dict[str, Union[str, list[str]]]) -> str:
  inp, label = '', ''
  example["options_ru"] = [option for option in example["options_ru"] if option]
  right_answer = example['options_ru'][option_id_dict[example['answer']]]

  right_answer = right_answer.replace('"', "'")

  inp += example['article_ru'] + " " + "ВОПРОС: Какое название лучше всего подойдёт для этого текста? "
  inp += f'ПРАВИЛЬНЫЙ ОТВЕТ: "{right_answer}".'
  inp += 'НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА: '

  options = example["options_ru"]
  options = [
      option.replace('"', "'") for option in options if option != right_answer
  ]
  options = [
      f'"{option}"' for option in options
  ]
  label = "; ".join(options)
  distractors_len = len(tokenizer(label)["input_ids"])
    
  return {
      "inp": inp,
      "distractors": label,
      "right_answer": right_answer,
      "distractors_len": distractors_len
  }

In [4]:
with open("title_dataset_pretty_filtered.json", 'r', encoding="utf8") as inp:
    title_dataset = json.load(inp)

(
    title_dataset_train,
    title_dataset_val,
    title_dataset_test
) = (
    title_dataset["train"],
    title_dataset["val"],
    title_dataset["test"]
)
title_dataset_train = Dataset.from_list(title_dataset_train)
title_dataset_val = Dataset.from_list(title_dataset_val)
title_dataset_test = Dataset.from_list(title_dataset_test)

title_dataset_train = title_dataset_train.map(to_new_format)
title_dataset_val = title_dataset_val.map(to_new_format)
title_dataset_test = title_dataset_test.map(to_new_format)

Map:   0%|          | 0/4375 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [5]:
pd.Series(title_dataset_train["distractors_len"]).describe()

count    4375.000000
mean       28.631771
std         7.996853
min        12.000000
25%        23.000000
50%        27.000000
75%        33.000000
max       133.000000
dtype: float64

In [6]:
MAX_LEN = pd.Series(title_dataset_train["distractors_len"]).quantile(0.99)
MAX_LEN

53.0

Evaluate on the whole dataset:

In [7]:
def get_metric_inputs_seq2seq(
    input_batch: list[str], #label_batch: list[str],
    model: PreTrainedModel, tokenizer: PreTrainedTokenizer
) -> list[str]:
    input_batch_ = tokenizer(
        input_batch,
        return_tensors="pt",
        padding=True
    )["input_ids"].to(tt.device("cuda:0"))
    # label_batch_ = tokenizer(label_batch, return_tensors="pt", padding=True)["input_ids"]

    # output_length = label_batch_.shape[-1]

    with tt.no_grad():
        output_batch = model.generate(input_batch_, max_length=MAX_LEN)

    output = [
        sent.replace("<pad>", " ").replace("</s>", " ").strip() for sent in tokenizer.batch_decode(
            output_batch)
    ]
    
    del input_batch_
    del output_batch
    # del label_batch_
    tt.cuda.empty_cache()

    return output

def compute_metrics(output: list[str], label_batch: list[str]) -> dict:
    metric_dict = {
        "bleu": bleu4.compute(predictions=output, references=[[label] for label in label_batch]),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

In [8]:
BATCH_SIZE = 1
N_STEPS = (len(title_dataset_test) // BATCH_SIZE) + 1

metrics = []

for i in tqdm_notebook(range(N_STEPS), total=N_STEPS):
    slice = title_dataset_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE]

    if slice["inp"]:
        output= get_metric_inputs_seq2seq(slice["inp"], model, tokenizer)

        distractors = [
            item.replace('\n', '').replace('  ',' ').replace('  ',' ').strip()
            for item in slice["distractors"]
        ]

        try:
            metric = compute_metrics(output, distractors)
            metrics.append({
                "article": slice["article_ru"][0],
                "right_answer": slice["right_answer"][0],
                "distractors": distractors[0],
                "output": output[0],
    
                "bleu": metric["bleu"]["bleu"],
                "sbleu": metric["sbleu"]["score"],
                "rouge1": metric["rouge"]["rouge1"],
                "rouge2": metric["rouge"]["rouge2"],
                "rougeL": metric["rouge"]["rougeL"],
                "rougeLsum": metric["rouge"]["rougeLsum"],
                "meteor": metric["meteor"]["meteor"],
    
                "article_orig": slice["article"][0],
                "question_orig": slice["question"][0],
                "options_orig": slice["options"][0],
                "right_answer_orig": slice["answer"][0]
            })
        except ZeroDivisionError:
            metrics.append({
                "article": slice["article_ru"][0],
                "right_answer": slice["right_answer"][0],
                "distractors": distractors[0],
                "output": output[0],
    
                "bleu": 0,
                "sbleu": 0,
                "rouge1": 0,
                "rouge2": 0,
                "rougeL": 0,
                "rougeLsum": 0,
                "meteor": 0,
    
                "article_orig": slice["article"][0],
                "question_orig": slice["question"][0],
                "options_orig": slice["options"][0],
                "right_answer_orig": slice["answer"][0]
            })

/tmp/ipykernel_8959/4159526788.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N_STEPS), total=N_STEPS):


  0%|          | 0/243 [00:00<?, ?it/s]

In [9]:
metrics[0]

{'article': 'Девушка с голубыми глазами - голубоглазый. Мужчина с длинными ногами - это человек с длинными ногами. Женщина с белыми волосами - женщина с белыми волосами. Дети, у которых хорошая внешность, красивые дети. Что ты делаешь, когда хочешь купить одежду? Ты идешь в магазин. Если ты найдёшь одежду подходящего размера для тебя, и если она будет готова, ты, вероятно, купишь ее. Их называют готовой одеждой. Если вы не сможете найти одежду того размера, вы пойдёте в портной. Портной - это человек, который делает одежду. Он измерит вас тщательно, а потом сделает для вас одеяние. Такая одежда называется ручной одеждой.\nКак мы называем человека, который плохо одет? Мы называем его плохо одетым человеком. Женщина, хорошо одетая, называется хорошо одетая женщина.\nЧто ты носишь, когда идет сильный дождь? Ты носишь пальто, которое удержит дождь. Такое пальто называется плащом. Она сделана из водонепроницаемой ткани - зуб, который не позволяет воде пройти. В Англии много дождя. Если прие

In [10]:
metrics = pd.DataFrame(metrics)

In [11]:
metrics.describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,242.0,242.000000,242.0,242.0,242.0,242.0,242.000000
mean,0.0,0.476918,0.0,0.0,0.0,0.0,0.013104
std,0.0,0.698826,0.0,0.0,0.0,0.0,0.022389
min,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
25%,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
50%,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
75%,0.0,0.896660,0.0,0.0,0.0,0.0,0.022624
max,0.0,2.847964,0.0,0.0,0.0,0.0,0.122275


In [12]:
metrics.to_excel("T5Metrics-Title-Baseline.xlsx", engine="openpyxl")